In [4]:
import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import*
import time


# model=YOLO(r"headdataset\best(office).pt")
model = YOLO(r'D:\headdatset\yolov5n.pt')

# peoplecount1
# area1=[(268, 472), (325, 467), (567, 583), (482, 593)]
# area2=[(343, 474), (574, 578), (604, 534), (434, 442)]

#shopping mall
# area1=[(839, 289), (806, 308), (1046, 391), (1072, 366)]
# area2=[(887, 286), (931, 260), (1122, 317), (1089, 348)]

# cctvdemo
area1= [(408, 112), (428, 193), (531, 190), (612, 106)]
area2 = [(417, 238), (652, 225), (687, 341), (387, 360)]
cv2.namedWindow('RGB')


# cap = cv2.VideoCapture('shoppingmall.mp4')
# cap = cv2.VideoCapture('cctvdemo.mp4')
cap = cv2.VideoCapture(1)
# cap = cv2.VideoCapture('peoplecount1.mp4')

# Read the class names for YOLO detections
my_file = open('coco.txt')
data = my_file.read()
class_list = data.split("\n")

count=0
people_entering={}
people_exiting={}
tracker = Tracker()
entering = set()
exiting = set()
n_p = 0


while True:    

    ret, frame = cap.read()
    if not ret:
        print("no ret")
        break
    if frame is None:
        print("Error: Received empty frame.")
        break
    frame=cv2.resize(frame,(1200,600))

    count += 1
    if count % 2 != 0:
        continue

    results = model.predict(frame)
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype("float")

    list = []
    conf_threshold = 0.5

    for index, row in px.iterrows():
        x1, y1, x2, y2, d = map(int, row[:5])
        c = class_list[d]
        if 'person' in c:
            # Draw the bounding box and label on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 255, 255), 2)
            cv2.putText(frame, c, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9,(255, 255, 255), 2)
            list.append([x1, y1, x2, y2])

   
    bbox_id = tracker.update(list)
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox

        # Check for entering the first area (area1)
        # results2 = cv2.pointPolygonTest(np.array(area1, np.int32), (x4, y4), False)
        results2 = cv2.pointPolygonTest(np.array(area1, np.int32), (x2, y2), False)

        if results2 >= 0 and id not in people_entering:
            people_entering[id] = (x4, y4)  # Mark as entering
            cv2.rectangle(frame, (x3, y3), (x4, y4), (255, 0, 0), 2) #draw blue

        # Check for entering the second area (area2)
        if id in people_entering:
            # results3 = cv2.pointPolygonTest(np.array(area2, np.int32), (x4, y4), False)
            results3 = cv2.pointPolygonTest(np.array(area2, np.int32), (x2, y2), False)

            if results3 >= 0 and id not in entering:  # Make sure it's only added once
                entering.add(id)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                cv2.circle(frame, (x4, y4), 4, (255, 0, 255), 1)
                cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)

        # Check for exiting the first area (area2)
        # results = cv2.pointPolygonTest(np.array(area2, np.int32), (x4, y4), False)
        results = cv2.pointPolygonTest(np.array(area2, np.int32),(x2, y2), False)

        if results >= 0 and id not in people_exiting:
            people_exiting[id] = (x4, y4)  # Mark as exiting
            cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 2) # Draw a red rectangle around the person

        # Check for exiting the second area (area1)
        if id in people_exiting:
            # results1 = cv2.pointPolygonTest(np.array(area1, np.int32), (x4, y4), False)
            results1 = cv2.pointPolygonTest(np.array(area1, np.int32), (x2, y2), False)

            if results1 >= 0 and id not in exiting:  # Make sure it's only added once
                exiting.add(id)
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                # cv2.circle(frame, (x4, y4), 4, (255, 0, 255), 1)
                # cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)

    cv2.polylines(frame, [np.array(area1, np.int32)], True, (255, 0, 0), 2)
    cv2.putText(frame, str('1'), (410, 531), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1)

    cv2.polylines(frame, [np.array(area2, np.int32)], True, (255, 0, 0), 2)
    cv2.putText(frame, str('2'), (466, 485), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0), 1)

    # Initialize number if not already initialized
    if 'number' not in locals():
        number = 0
        
    if 'number1' not in locals():
        number1 = 0

    i = len(entering)
    o = len(exiting)

    if i:
        number += 1
    if o:
        number1 += 1

    # Display the number of people entering
    cv2.putText(frame, "Entering: " + str(i), (60, 80), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 1)

    # Display the number of people exiting
    cv2.putText(frame, "Exiting: " + str(o), (60, 140), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 1)

    # Display the total number of people
    # cv2.putText(frame, "No of people: " + str(number), (100, 200), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 0, 255), 1)


    # Display the frame with bounding boxes and count
    cv2.imshow("RGB", frame)
    if cv2.waitKey(3) & 0xFF == 27:
        break
    

cap.release()
cv2.destroyAllWindows()



PRO TIP  Replace 'model=D:\headdatset\yolov5n.pt' with new 'model=D:\headdatset\yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



no ret


In [1]:
import cv2

from ultralytics import YOLO, solutions

model = YOLO("yolov8s.pt")
cap = cv2.VideoCapture("cctvdemo.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define line points
line_points = [(325, 467), (567, 583)]
classes_to_count = [0]

# Video writer
video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init Object Counter
counter = solutions.ObjectCounter(
    view_img=True,
    reg_pts=line_points,
    classes_names=model.names,
    draw_tracks=True,
    line_thickness=2,
)

while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    tracks = model.track(im0, persist=True, show=False)

    im0 = counter.count_objects(im0, tracks)
    video_writer.write(im0)

cap.release()
video_writer.release()
cv2.destroyAllWindows()

Ultralytics Solutions:  {'region': None, 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'view_img': True, 'reg_pts': [(325, 467), (567, 583)], 'classes_names': {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50

TypeError: ObjectCounter.count_objects() missing 2 required positional arguments: 'prev_position' and 'cls'

In [3]:
import cv2

from ultralytics import solutions

cap = cv2.VideoCapture("shoppingmall.mp4")
assert cap.isOpened(), "Error reading video file"
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Define region points
region_points = [(931, 260), (1122, 317)]  # For line counting
# region_points = [(20, 400), (1080, 400), (1080, 360), (20, 360)]  # For rectangle region counting
# region_points = [(20, 400), (1080, 400), (1080, 360), (20, 360), (20, 400)]  # For polygon region counting

# Video writer
# video_writer = cv2.VideoWriter("object_counting_output.avi", cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

# Init ObjectCounter
counter = solutions.ObjectCounter(
    show=True,  # Display the output
    region=region_points,  # Pass region points
    model="yolo11n.pt",  # model="yolo11n-obb.pt" for object counting using YOLO11 OBB model.
    classes=[0],  # If you want to count specific classes i.e person and car with COCO pretrained model.
    show_in=True,  # Display in counts
    show_out=True,  # Display out counts
    # line_width=2,  # Adjust the line width for bounding boxes and text display
)

# Process video
while cap.isOpened():
    success, im0 = cap.read()
    if not success:
        print("Video frame is empty or video processing has been successfully completed.")
        break
    im0=cv2.resize(im0,(1200,600))
    im0 = counter.count(im0)
    # video_writer.write(im0)

cv2.imshow("RGB", im0)
if cv2.waitKey(3) & 0xFF == 27:
    break

cap.release()
# video_writer.release()
cv2.destroyAllWindows()

Ultralytics Solutions:  {'region': [(931, 260), (1122, 317)], 'show_in': True, 'show_out': True, 'colormap': None, 'up_angle': 145.0, 'down_angle': 90, 'kpts': [6, 8, 10], 'analytics_type': 'line', 'json_file': None, 'view_img': True, 'reg_pts': [(325, 467), (567, 583)], 'classes_names': {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwi

KeyboardInterrupt: 